In [41]:
import numpy as np
import cvxpy as cp

In [45]:
# The amount of energy output of
# coal, natural gas, nuclear, hydroelectric, petroleum
# MW-hr
energy_output = cp.Variable(5, integer=True)

# Unit cost of generation
# dollor / MW-hr
unit_cost = np.array([6.0, 5.5, 4.5, 5.0, 7.0])

total_capacity = np.array([45000, 15000, 45000, 24000, 48000])

# Rates of Pollutant Emissions
# g / MW-hr
pollutant_emissions = np.array([
    [1.5, 1.2, 0.7, 0.4],
    [0.2, 0.5, 0, 0],
    [0.5, 0.2, 0.4, 0.5],
    [0, 0, 0, 0],
    [0.4, 0.8, 0.5, 0.1]
]).T

# Alowable Pollution
# g
limits_on_pollutant_emissions = np.array([75000, 60000, 30000, 25000])

In [67]:
objective_function = cp.Minimize(unit_cost.T @ energy_output)

# resource1 = energy_output >= 0
resource2 = energy_output >= np.array([36000, 0, 0, 0, 0])
resource3 = energy_output <= total_capacity


# regulation 1: nuclear material energy should not exceed 20% of total energy
# E_nuclear <= 0.2 * sum(E)
regulation1 = np.array([-0.2, -0.2, 0.8, -0.2, -0.2]) @ energy_output <= 0 

# regulation 2: at least 80% capacity of coal plant is used
# E_coal >= 0.8 * coal
regulation2 = np.array([1.25, 0, 0, 0, 0]) @ energy_output >= total_capacity[0]

# regulation 3: energy from natural gas should be at least 30% or more on that from petroleum
# E_gas >= 0.3 * E_petro
regulation3 = np.array([0, 1, 0,0, -0.3]) @ energy_output >= 0

# regulation 4: the pollutant produced must meet regulations
# 
regulation4 = pollutant_emissions @ energy_output <= limits_on_pollutant_emissions

# need 1: A total of 100000MW
need1 = sum(energy_output) >= 125000

In [68]:
constraints = [
    regulation1,
    regulation2,
    regulation3,
    regulation4,
    need1,
    resource2,
    resource3
]

prob = cp.Problem(objective_function, constraints)

In [69]:
prob.solve()

inf

In [70]:
energy_output.value